# Name: Dhanush Kobal
# Project: Heart Prediction failure 

## We will use AutoML to create some ML models that Azure thinks it's best

In [1]:
# uploading the packages
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

In [2]:
ws = Workspace.from_config()
experiment_name = 'Heart_prediction_AutoML'

experiment=Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
Subscription ID,7ee37d15-c46a-4653-9184-c5d6f9009da9
Workspace,ex-1-workspace
Resource Group,example-1
Location,eastus2
Experiment Name,Heart_prediction_AutoML


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# connecting to our CPU Cluster
cpu_cluster_name = "my-compute"
compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)

In [4]:
# initalizing our data into the database
from azureml.core import Workspace, Dataset, Datastore

datastore = Datastore.get_default(ws)
my_datastore_name = 'workspaceblobstore'
my_datastore = Datastore.get(ws, my_datastore_name)


dataset_name_train = "Heart_data_train" # the dataset you uploaded into Azure
train_data = Dataset.get_by_name(ws, dataset_name_train, version='latest')
train_data = train_data.to_pandas_dataframe()


dataset_name_test = "Heart_data_test" # the dataset you uploaded into Azure
test_data = Dataset.get_by_name(ws, dataset_name_test, version='latest')
test_data = test_data.to_pandas_dataframe()

In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 19 columns):
Column1                     209 non-null int64
age                         209 non-null float64
anaemia                     209 non-null object
creatinine_phosphokinase    209 non-null int64
diabetes                    209 non-null object
ejection_fraction           209 non-null int64
high_blood_pressure         209 non-null object
platelets                   209 non-null float64
serum_creatinine            209 non-null float64
serum_sodium                209 non-null int64
sex                         209 non-null object
smoking                     209 non-null object
time                        209 non-null int64
age_cat                     209 non-null object
age_death_event             209 non-null object
ejection_death_event        209 non-null object
platelets_death_event       209 non-null object
serum_death_event           209 non-null object
y_train                     209 n

In [6]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 18 columns):
Column1                     90 non-null int64
age                         90 non-null float64
anaemia                     90 non-null object
creatinine_phosphokinase    90 non-null int64
diabetes                    90 non-null object
ejection_fraction           90 non-null int64
high_blood_pressure         90 non-null object
platelets                   90 non-null float64
serum_creatinine            90 non-null float64
serum_sodium                90 non-null int64
sex                         90 non-null object
smoking                     90 non-null object
time                        90 non-null int64
age_cat                     90 non-null object
age_death_event             90 non-null object
ejection_death_event        90 non-null object
platelets_death_event       90 non-null object
serum_death_event           90 non-null object
dtypes: float64(3), int64(5), object(10)
memory usage

In [7]:
train_data = train_data.drop('Column1', axis = 1)
test_data = test_data.drop('Column1', axis = 1)

In [8]:
train_data.head()
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 18 columns):
age                         209 non-null float64
anaemia                     209 non-null object
creatinine_phosphokinase    209 non-null int64
diabetes                    209 non-null object
ejection_fraction           209 non-null int64
high_blood_pressure         209 non-null object
platelets                   209 non-null float64
serum_creatinine            209 non-null float64
serum_sodium                209 non-null int64
sex                         209 non-null object
smoking                     209 non-null object
time                        209 non-null int64
age_cat                     209 non-null object
age_death_event             209 non-null object
ejection_death_event        209 non-null object
platelets_death_event       209 non-null object
serum_death_event           209 non-null object
y_train                     209 non-null bool
dtypes: bool(1), float64(3), int64

In [9]:
test_data.head()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 17 columns):
age                         90 non-null float64
anaemia                     90 non-null object
creatinine_phosphokinase    90 non-null int64
diabetes                    90 non-null object
ejection_fraction           90 non-null int64
high_blood_pressure         90 non-null object
platelets                   90 non-null float64
serum_creatinine            90 non-null float64
serum_sodium                90 non-null int64
sex                         90 non-null object
smoking                     90 non-null object
time                        90 non-null int64
age_cat                     90 non-null object
age_death_event             90 non-null object
ejection_death_event        90 non-null object
platelets_death_event       90 non-null object
serum_death_event           90 non-null object
dtypes: float64(3), int64(4), object(10)
memory usage: 12.1+ KB


In [10]:
from azureml.core import Workspace, Dataset, Datastore

datastore = Datastore.get_default(ws)
my_datastore_name = 'workspaceblobstore'
my_datastore = Datastore.get(ws, my_datastore_name)

Dataset.Tabular.register_pandas_dataframe(train_data,datastore,'Heart_data_train_1')
Dataset.Tabular.register_pandas_dataframe(test_data,datastore,'Heart_data_test_1')

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/bd39f776-7e46-4ee5-ac77-06a9c6a3cbfd/
Successfully uploaded file to datastore.
Creating and registering a new dataset.


Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Successfully created and registered a new dataset.
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/9b7caea0-e22c-4276-b9af-bb9c7addb062/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


{
  "source": [
    "('workspaceblobstore', 'managed-dataset/9b7caea0-e22c-4276-b9af-bb9c7addb062/')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "9bdcdbf1-b41c-4a66-b1c3-d87359ca752f",
    "name": "Heart_data_test_1",
    "version": 1,
    "workspace": "Workspace.create(name='ex-1-workspace', subscription_id='7ee37d15-c46a-4653-9184-c5d6f9009da9', resource_group='example-1')"
  }
}

In [11]:
# open the train data
dataset_name = "Heart_data_train_1" # the dataset you uploaded into Azure
train_data = Dataset.get_by_name(ws, dataset_name, version='latest')
train_data.to_pandas_dataframe().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 18 columns):
age                         209 non-null float64
anaemia                     209 non-null object
creatinine_phosphokinase    209 non-null int64
diabetes                    209 non-null object
ejection_fraction           209 non-null int64
high_blood_pressure         209 non-null object
platelets                   209 non-null float64
serum_creatinine            209 non-null float64
serum_sodium                209 non-null int64
sex                         209 non-null object
smoking                     209 non-null object
time                        209 non-null int64
age_cat                     209 non-null object
age_death_event             209 non-null object
ejection_death_event        209 non-null object
platelets_death_event       209 non-null object
serum_death_event           209 non-null object
y_train                     209 non-null bool
dtypes: bool(1), float64(3), int64

In [12]:
label_column_name = 'y_train'

automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": 'AUC_weighted',
    "enable_early_stopping": True,
    "max_concurrent_iterations": 2, # This is a limit for testing purpose, please increase it as per cluster size
    "experiment_timeout_hours": 0.25, # This is a time limit for testing purposes, remove it for real use cases, this will drastically limit ablity to find the best model possible
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             compute_target = compute_target,
                             training_data = train_data,
                             label_column_name = label_column_name,
                             enable_stack_ensemble=True,
                             enable_voting_ensemble=True,
                             **automl_settings
                            )

In [13]:
remote_run = experiment.submit(automl_config, show_output = False)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Heart_prediction_AutoML,AutoML_26088775-977d-493c-87b1-c1d800a399a7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [15]:
best_run, fitted_model = remote_run.get_output()
fitted_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mn...
), random_state=None, reg_alpha=0.6842105263157894, reg_lambda=0.15789473684210525, subsample=1))], verbose=False))], flatten_transform=None, weights=[0.13333333333333333, 0.06666666666666667, 0.13333333333333333, 0.06666666666666667, 0.13333333333333333, 0.13333333333333333, 0.06666666666666667, 0.26666666666666666]))],
                                       'verbose': False},
                             y_transformer={},
                             y_transformer_name='LabelEncoder')

In [16]:
# register the model
description = 'Best AutoML for Heart diease'
tags = {'project' : "Heart diease", "creator" : "Dhanush Kobal",
       'metric': 'AUC_weighted'}
model_name = 'Heart_prediction_automl'

remote_run.register_model(model_name=model_name,
                          description=description, tags=tags)

Model(workspace=Workspace.create(name='ex-1-workspace', subscription_id='7ee37d15-c46a-4653-9184-c5d6f9009da9', resource_group='example-1'), name=Heart_prediction_automl, id=Heart_prediction_automl:7, version=7, tags={'project': 'Heart diease', 'creator': 'Dhanush Kobal', 'metric': 'AUC_weighted'}, properties={})

In [17]:
# we will now upload the test data and predict values
dataset_name_test = "Heart_data_test_1" # the dataset you uploaded into Azure
test_data = Dataset.get_by_name(ws, dataset_name_test, version='latest')

In [20]:
train_data.to_pandas_dataframe().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 18 columns):
age                         209 non-null float64
anaemia                     209 non-null object
creatinine_phosphokinase    209 non-null int64
diabetes                    209 non-null object
ejection_fraction           209 non-null int64
high_blood_pressure         209 non-null object
platelets                   209 non-null float64
serum_creatinine            209 non-null float64
serum_sodium                209 non-null int64
sex                         209 non-null object
smoking                     209 non-null object
time                        209 non-null int64
age_cat                     209 non-null object
age_death_event             209 non-null object
ejection_death_event        209 non-null object
platelets_death_event       209 non-null object
serum_death_event           209 non-null object
y_train                     209 non-null bool
dtypes: bool(1), float64(3), int64

In [21]:
test_data.to_pandas_dataframe().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 17 columns):
age                         90 non-null float64
anaemia                     90 non-null object
creatinine_phosphokinase    90 non-null int64
diabetes                    90 non-null object
ejection_fraction           90 non-null int64
high_blood_pressure         90 non-null object
platelets                   90 non-null float64
serum_creatinine            90 non-null float64
serum_sodium                90 non-null int64
sex                         90 non-null object
smoking                     90 non-null object
time                        90 non-null int64
age_cat                     90 non-null object
age_death_event             90 non-null object
ejection_death_event        90 non-null object
platelets_death_event       90 non-null object
serum_death_event           90 non-null object
dtypes: float64(3), int64(4), object(10)
memory usage: 12.1+ KB


In [22]:
y_pred = fitted_model.predict(test_data.to_pandas_dataframe())

In [23]:
y_pred

array([False, False,  True, False, False, False, False, False, False,
       False, False, False,  True, False,  True, False, False, False,
       False,  True, False,  True, False,  True, False,  True, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False,  True, False,  True, False,
        True, False, False,  True, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False,  True, False,  True,  True, False, False, False,  True,
       False, False, False, False, False,  True, False,  True,  True])

In [24]:
pd.DataFrame({ 'AutoML_prediction' :y_pred}).to_csv('AutoML_Prediction')